# ML.Net - IrisFlowerClustering


In [1]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML version 1.5.0

In [2]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;

In [3]:
/// A data transfer class that holds a single iris flower.
public class IrisData
{
    [LoadColumn(0)] public float SepalLength;
    [LoadColumn(1)] public float SepalWidth;
    [LoadColumn(2)] public float PetalLength;
    [LoadColumn(3)] public float PetalWidth;
    [LoadColumn(4)] public string Label;  // ignored during training
}

/// A prediction class that holds a single cluster prediction.
public class ClusterPrediction
{
    [ColumnName("PredictedLabel")] public uint PredictedClusterId;
    [ColumnName("Score")] public float[] Distances;
}    

In [4]:
static readonly string _dataPath = Path.Combine(Environment.CurrentDirectory, "Datasets\\Iris_Data", "iris-data.txt");
static readonly string _modelPath = Path.Combine(Environment.CurrentDirectory, "Datasets\\Iris_Data", "IrisClusteringModel.zip");

var mlContext = new MLContext(seed: 0);
        
// read the iris flower data from a text file
var trainingData = mlContext.Data.LoadFromTextFile<IrisData>(
    path: _dataPath, 
    hasHeader: false, 
    separatorChar: ',');

// set up a learning pipeline
// step 1: concatenate input features into a single column
var pipeline = mlContext.Transforms.Concatenate(
    "Features", 
    "SepalLength", 
    "SepalWidth", 
    "PetalLength", 
    "PetalWidth")

    // step 2: use the k-means clustering algorithm
    // assume there are 3 flower types
    .Append(mlContext.Clustering.Trainers.KMeans(
        "Features",
        numberOfClusters: 3));

// train the model on the data file
Console.WriteLine("Start training model....");
var model = pipeline.Fit(trainingData);
Console.WriteLine("Model training complete!");

//Save Model
using (var fileStream = new FileStream(_modelPath, FileMode.Create, FileAccess.Write, FileShare.Write))
{
    mlContext.Model.Save(model, trainingData.Schema, fileStream);
}

// predict a single flower based on input data
Console.WriteLine("Predicting a sample flower....");
var prediction = mlContext.Model.CreatePredictionEngine<IrisData, ClusterPrediction>(model).Predict(
    new IrisData()
    {
        SepalLength = 3.3f,
        SepalWidth = 1.6f,
        PetalLength = 0.2f,
        PetalWidth = 5.1f,
    });

Console.WriteLine($"Cluster: {prediction.PredictedClusterId}");
Console.WriteLine($"Distances: {string.Join(" ", prediction.Distances)}");

Start training model....
Model training complete!
Predicting a sample flower....
Cluster: 2
Distances: 54,509613 31,393993 38,95703
